In [1]:
import pandas as pd
import re
import datetime
import numpy as np

# 2019 Data Preprocess

In [98]:
pbp_data_19 = pd.read_csv("./pbp_data/regular_season/reg_pbp_2019.csv")

/var/folders/08/6rx4zjtn10557fvbrhbl5ykw0000gn/T/ipykernel_89607/20673864.py:1: DtypeWarning: Columns (167,168,169,170,175,176,179,180,183,184,189,190,191,192,195,196,204,205,206,219,220,221,222,223,224,227,228,232,233,234,239,241,242) have mixed types. Specify dtype option on import or set low_memory=False.
  pbp_data_19 = pd.read_csv("./pbp_data/regular_season/reg_pbp_2019.csv")


In [74]:
highlights19 = pd.read_csv("./2019_NFL_highlight_transcript - Sheet1.csv")
highlights19 = highlights19[~highlights19["inning"].isna()]

In [75]:
relevant_cols = pbp_data_19.drop(columns=(list(pbp_data_19.iloc[:,58:110].columns) + ["desc","home_timeouts_remaining","home_timeouts_remaining","timeout","timeout_team","play_id","game_id","game_date"])).columns

In [76]:
pbp_data_19.columns

Index(['play_id', 'game_id', 'home_team', 'away_team', 'posteam',
       'posteam_type', 'defteam', 'side_of_field', 'yardline_100', 'game_date',
       ...
       'penalty_player_id', 'penalty_player_name', 'penalty_yards',
       'replay_or_challenge', 'replay_or_challenge_result', 'penalty_type',
       'defensive_two_point_attempt', 'defensive_two_point_conv',
       'defensive_extra_point_attempt', 'defensive_extra_point_conv'],
      dtype='object', length=256)

In [77]:
def process_inning_time(time_str):
    quarter = time_str[0]
    if quarter == "O":
        time_remaining = 0
    else:
        time_remaining = (4 - int(quarter)) * 900
    times = re.findall("\d?\d:\d\d",time_str)
    timedeltas = pd.to_timedelta(pd.to_datetime(times,format='%M:%S').strftime('%H:%M:%S').values)
    timelists = (timedeltas + datetime.timedelta(seconds=time_remaining)).seconds
    return pd.Interval(timelists[1],timelists[0],closed="both")

In [78]:
highlights19["intervals"] = highlights19["inning"].apply(process_inning_time)
highlights19["date"] = pd.to_datetime(highlights19["date"])
pbp_data_19["game_date"] = pd.to_datetime(pbp_data_19["game_date"])

In [79]:
merged19 = pbp_data_19.merge(highlights19,left_on=["home_team","away_team","game_date"], right_on=["home","away","date"])

In [80]:
def in_interval(row):
    return row["game_seconds_remaining"] in row["intervals"]

In [81]:
plays19 = merged19[merged19.apply(lambda x: x["game_seconds_remaining"] in x["intervals"],axis=1)]

In [82]:
touches = plays19[plays19["touchdown"] == 1].groupby("intervals")['touchdown'].sum()

In [83]:
mi_index = touches[touches == 1].index

In [84]:
plays19 = plays19[(plays19["touchdown"] == 1) & (plays19["intervals"].isin(mi_index))]

In [85]:
X = plays19[relevant_cols.to_list()+["transcript"]]

In [86]:
# transcript_grouped = X.groupby("transcript").groups
# transcript_groups = list(transcript_grouped.keys())

In [94]:
ret_list = []
for i in range(X.shape[0]):
    ret_list += [{"json":X[relevant_cols].iloc[i].replace(0,np.nan).dropna().to_json(),
                  "transcript":X["transcript"].iloc[i]}]

In [95]:
ret_list[]

[{'json': '{"home_team":"TB","away_team":"NYG","posteam":"TB","posteam_type":"home","defteam":"NYG","side_of_field":"NYG","yardline_100":21.0,"quarter_seconds_remaining":594.0,"half_seconds_remaining":1494.0,"game_seconds_remaining":3294.0,"game_half":"Half1","drive":1,"sp":1,"qtr":1,"down":3.0,"time":"09:54","yrdln":"NYG 21","ydstogo":6,"ydsnet":75,"play_type":"pass","yards_gained":21,"shotgun":1,"qb_dropback":1.0,"pass_length":"deep","pass_location":"right","air_yards":21.0,"away_timeouts_remaining":3,"td_team":"TB","posteam_timeouts_remaining":3.0,"defteam_timeouts_remaining":3.0,"total_home_score":6,"posteam_score_post":6.0,"score_differential_post":6.0,"first_down_pass":1.0,"third_down_converted":1.0,"qb_hit":1.0,"pass_attempt":1.0,"touchdown":1.0,"pass_touchdown":1.0,"complete_pass":1.0,"passer_player_id":"00-0031503","passer_player_name":"J.Winston","receiver_player_id":"00-0031408","receiver_player_name":"M.Evans","qb_hit_1_player_id":"00-0033883","qb_hit_1_player_name":"J.Pepp

In [21]:
ret_list = []
for i in range(len(transcript_groups)):
    transcript_group = X.loc[transcript_grouped[transcript_groups[i]]]
    ret_list += [{"json":transcript_group.loc[transcript_group["touchdown"]==1,relevant_cols].replace(0,np.nan).dropna(axis=1).iloc[0].to_json(),
                  "transcript":transcript_group["transcript"].iloc[0]}]

In [24]:
ret_list[4]["transcript"]

"Second and goal, play action, prescott finds the target it's waiting for the touchdown! Jason Witten has another thanksgiving touchdown!"

In [105]:
np.random.sample(np.array(ret_list))

ValueError: maximum supported dimension for an ndarray is 32, found 33

In [123]:
my_sample = pd.Series(ret_list).sample(4,random_state=82)
my_sample

6     {'json': '{"home_team":"TB","away_team":"NYG",...
26    {'json': '{"home_team":"SF","away_team":"SEA",...
28    {'json': '{"home_team":"WAS","away_team":"DET"...
12    {'json': '{"home_team":"BAL","away_team":"CLE"...
dtype: object

In [127]:
my_sample.iloc[3]

{'json': '{"home_team":"BAL","away_team":"CLE","posteam":"BAL","posteam_type":"home","defteam":"CLE","side_of_field":"MID","yardline_100":50.0,"quarter_seconds_remaining":40.0,"half_seconds_remaining":40.0,"game_seconds_remaining":40.0,"game_half":"Half2","drive":23,"sp":1,"qtr":4,"down":1.0,"time":"00:40","yrdln":"MID 50","ydstogo":15,"ydsnet":79,"play_type":"pass","yards_gained":50,"shotgun":1,"no_huddle":1,"qb_dropback":1.0,"pass_length":"short","pass_location":"left","air_yards":6.0,"yards_after_catch":44.0,"away_timeouts_remaining":2,"td_team":"BAL","defteam_timeouts_remaining":2.0,"total_home_score":24,"total_away_score":40,"posteam_score":18.0,"defteam_score":40.0,"score_differential":-22.0,"posteam_score_post":24.0,"defteam_score_post":40.0,"score_differential_post":-16.0,"first_down_pass":1.0,"pass_attempt":1.0,"touchdown":1.0,"pass_touchdown":1.0,"complete_pass":1.0,"passer_player_id":"00-0034796","passer_player_name":"L.Jackson","receiver_player_id":"00-0030663","receiver_pl

In [129]:
pd.Series(ret_list).sample(1,random_state=57).iloc[0]

{'json': '{"home_team":"BAL","away_team":"CLE","posteam":"CLE","posteam_type":"away","defteam":"BAL","side_of_field":"BAL","yardline_100":2.0,"quarter_seconds_remaining":58.0,"half_seconds_remaining":958.0,"game_seconds_remaining":958.0,"game_half":"Half2","drive":14,"sp":1,"qtr":3,"down":1.0,"goal_to_go":1.0,"time":"00:58","yrdln":"BAL 02","ydstogo":2,"ydsnet":74,"play_type":"run","yards_gained":2,"run_location":"left","run_gap":"tackle","away_timeouts_remaining":3,"td_team":"CLE","posteam_timeouts_remaining":3.0,"defteam_timeouts_remaining":3.0,"total_home_score":10,"total_away_score":23,"posteam_score":17.0,"defteam_score":10.0,"score_differential":7.0,"posteam_score_post":23.0,"defteam_score_post":10.0,"score_differential_post":13.0,"first_down_rush":1.0,"rush_attempt":1.0,"touchdown":1.0,"rush_touchdown":1.0,"rusher_player_id":"00-0034791","rusher_player_name":"N.Chubb"}',
 'transcript': 'Nick Chubb trying ot find the endzone and he does! Touchdown Cleveland.'}